# Mock data
This notebook prepares some mock data to be used in the GUI demonstrator>

In [2]:
from aurora.schemas.data_schemas import BatterySpecs, BatteryComposition, BatteryCapacity, BatteryMetadata, BatterySample
import datetime
import json, random
import pandas as pd
import numpy as np
import names

In [3]:
STD_SPECS = pd.read_csv('sample_specs.csv', dtype=str)
STD_SPECS

,manufacturer,composition.description,capacity.nominal,capacity.units,form_factor
0,BIG-MAP,C | LP57 | NMC811,1.539,mAh,2032
1,BIG-MAP,C | LP57 | LNO,1.539,mAh,2032
2,BIG-MAP,Si-C | LP57 | NMC811,1.539,mAh,2032
3,BIG-MAP,Si-C | LNMO | NMC622,1.539,mAh,2032
4,BIG-MAP,C | LP57 | NMC811,4.618,mAh,2032
5,BIG-MAP,C | LP57 | LNO,4.618,mAh,2032
6,BIG-MAP,Si-C | LP57 | NMC811,4.618,mAh,2032
7,BIG-MAP,Si-C | LNMO | NMC622,4.618,mAh,2032
8,Conrad energy,Li-based,45,mAh,2032
9,Conrad energy,Li-based,60,mAh,2032


In [4]:
STD_SPECS.keys()

Index(['manufacturer', 'composition.description', 'capacity.nominal',
       'capacity.units', 'form_factor'],
      dtype='object')

In [5]:
for key in STD_SPECS.keys():
    print(f'{key:14}:  {STD_SPECS.get(key).unique().tolist()}')

manufacturer  :  ['BIG-MAP', 'Conrad energy', 'Maxell']
composition.description:  ['C | LP57 | NMC811', 'C | LP57 | LNO', 'Si-C | LP57 | NMC811', 'Si-C | LNMO | NMC622', 'Li-based', 'LiMgO2-based']
capacity.nominal:  ['1.539', '4.618', '45', '60']
capacity.units:  ['mAh']
form_factor   :  ['2032', '2016']


In [6]:
def random_datetime(
        start=datetime.datetime(2020, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc),
        end=datetime.datetime.now(tz=datetime.timezone.utc)):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())))

In [39]:
def trim_string(s):
    "Remove special characters from a string"
    return s.translate({ord(c): None for c in ' -!@#$|'})

In [40]:
def create_random_sample():
    # pick one of the available specs
    random_spec = STD_SPECS.sample().iloc[0]
    battery_id = random.randint(0, 1000)
    dt = random_datetime()
    sample = BatterySample(
        manufacturer = random_spec.manufacturer,
        composition = BatteryComposition(description=random_spec.get('composition.description')),
        form_factor = random_spec.form_factor,
        capacity = BatteryCapacity(nominal=float(random_spec.get('capacity.nominal')), units=random_spec.get('capacity.units')),
        metadata = BatteryMetadata(
            # name = f'{names.get_first_name()}_{battery_id}',
            name = f"{battery_id}_{trim_string(random_spec.manufacturer)}_{trim_string(dt.date().isoformat())}",
            creation_datetime=dt,
            creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'
        ),
        battery_id = battery_id,
    )
    return sample

Now we generate 20 random samples and save the to a JSON file.

In [41]:
samples = [create_random_sample() for _ in range(40)]
samples = sorted(samples, key=lambda s: s.battery_id)

In [43]:
class BatterySampleEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, datetime.datetime):
                return obj.isoformat()
            else:
                super().default(obj)

json.dump([s.dict() for s in samples], open('available_samples.json', 'w'), cls=BatterySampleEncoder)
json.dump({s.battery_id: s.dict() for s in samples}, open('available_samples_id.json', 'w'), cls=BatterySampleEncoder)

In [44]:
[s.battery_id for s in samples]

[3,
 33,
 44,
 56,
 121,
 128,
 137,
 171,
 306,
 313,
 358,
 387,
 387,
 394,
 442,
 479,
 498,
 513,
 529,
 571,
 582,
 583,
 585,
 600,
 638,
 654,
 664,
 666,
 682,
 687,
 719,
 736,
 743,
 747,
 758,
 769,
 862,
 886,
 945,
 968]

Reload

In [45]:
data = json.load(open('available_samples.json', 'r'))
new_samples = [BatterySample.parse_obj(dic) for dic in data]

In [46]:
s = new_samples[0]
s

BatterySample(manufacturer='BIG-MAP', composition=BatteryComposition(description='Si-C | LNMO | NMC622', cathode='Si-C', anode='NMC622', electrolyte='LNMO'), form_factor='2032', capacity=BatteryCapacity(nominal=1.539, actual=None, units='mAh'), battery_id=3, metadata=BatteryMetadata(name='3_BIGMAP_20220309', creation_datetime=datetime.datetime(2022, 3, 9, 7, 10, 22, tzinfo=datetime.timezone.utc), creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'))